In [3]:
import pandas as pd 
import os
import shutil

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input

# Read and split the data 

In [5]:
df=pd.read_csv('last_mod.csv')

In [6]:
replace_dict = {'Pnemonia':1,
                'Normal':0}
df['Label'] = df['Label'].replace(replace_dict)

In [7]:
train_df = df[df.Dataset_type=='TRAIN']
test_df = df[df.Dataset_type=='TEST']

In [8]:
train_pneumonia_df = train_df[train_df.Label==1]
train_normal_df = train_df[train_df.Label==0]
test_pneumonia_df = test_df[test_df.Label==1]
test_normal_df = test_df[test_df.Label==0]

In [9]:
num_train_p= len(train_pneumonia_df)
num_train_n = len(train_normal_df)

t_n_train = num_train_p+num_train_n

#Take 10% from train to be validation

nval_p = round(0.1*num_train_p)
nval_n = round(0.1*num_train_n)

print(nval_p)
print(nval_n)

415
134


In [10]:
val_pneumonia_df = train_pneumonia_df[0:nval_p]
train_pneumonia_df = train_pneumonia_df[nval_p:num_train_p]

val_normal_df = train_normal_df[0:nval_n]
train_normal_df = train_normal_df[nval_n:]

In [11]:
val_images_pneumonia = val_pneumonia_df.X_ray_image_name.values.tolist()
val_images_normal = val_normal_df.X_ray_image_name.values.tolist()


In [12]:
def create_directory():
    try:
        os.makedirs('Coronahack-Chest-XRay-Dataset/val/Pneumonia')
        os.makedirs('Coronahack-Chest-XRay-Dataset/val/Normal')
    except:
        pass

create_directory()

In [13]:
train_path='Coronahack-Chest-XRay-Dataset/train'
test_path='Coronahack-Chest-XRay-Dataset/test'
# for image in val_images_pneumonia:
#     val_image_pneumonia = os.path.join(train_path, str(image))
#     shutil.copy(val_image_pneumonia, 'Coronahack-Chest-XRay-Dataset/val/Pneumonia')
    
# for image in val_images_normal:
#     val_image_normal = os.path.join(train_path, str(image))
#     shutil.copy(val_image_normal, 'Coronahack-Chest-XRay-Dataset/val/Normal')
    

In [14]:
batch_size = 32
train_data_generation = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_generator = train_data_generation.flow_from_directory('Coronahack-Chest-XRay-Dataset/train',
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('Coronahack-Chest-XRay-Dataset/val',
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

Found 4157 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [15]:
#Creating the model
base_model = InceptionV3(include_top=False, weights='imagenet', pooling='max', input_shape=(224,224,3))

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = Dropout(0.3)(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(0.3)(x)
x = Dense(512, activation = "relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation = "sigmoid")(x)

model = keras.Model(inputs,outputs)

base_model.trainable = False
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

The following Variables were used a Lambda layer's call (tf.nn.convolution), but
are not present in its tracked objects:
  <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 32) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization/beta:0' shape=(32,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_1), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_1/kernel:0' shape=(3, 3, 32, 32) dtype=float32>
It is possible t

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf.nn.convolution (TFOpLambda)  (None, 111, 111, 32) 0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.compat.v1.nn.fused_batch_nor ((None, 111, 111, 32 0           tf.nn.convolution[0][0]          
__________________________________________________________________________________________________
tf.nn.relu (TFOpLambda)         (None, 111, 111, 32) 0           tf.compat.v1.nn.fused_batch_norm[
______________________________________________________________________________________________

In [18]:
model.save('my_model.h5')

TypeError: Layer tf.nn.convolution was passed non-JSON-serializable arguments. Arguments had types: {'filters': <class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>, 'strides': [<class 'int'>, <class 'int'>], 'padding': <class 'str'>, 'dilations': [<class 'int'>, <class 'int'>], 'data_format': <class 'str'>, 'name': <class 'str'>}. They cannot be serialized out when saving the model.

In [ ]:
model.fit(train_generator,
          steps_per_epoch=train_generator.samples//batch_size,
          epochs = 10,
          validation_data=valid_generator,
          validation_steps=valid_generator.samples//batch_size)


base_model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(lr=0.00001),loss='binary_crossentropy',metrics=['accuracy'])

temp = valid_generator.samples//batch_size

model.fit(train_generator,
          steps_per_epoch=train_generator.samples//batch_size,
          epochs = 5,
          validation_data=valid_generator,
          validation_steps=temp)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_path,
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

model.evaluate(test_generator)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np 

In [ ]:
cm = confusion_matrix(y_true=test_generator.classes, y_pred=np.argmax(train_generator, axis=-1))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
test_generator.class_indices
cm_plot_labels = ['n', 'p']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion matrix')